In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from math import *
geolocator = Nominatim(user_agent="Problem1")
import numpy as np
import folium
from bs4 import BeautifulSoup
import requests
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
import re
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import __version__
import plotly
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from plotly.graph_objs import Scatter
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

# PROBLEM 1

Finding latitude and longitude of each city

In [3]:
names=['Kuala Lumpur','Jakarta','Bangkok','Hong Kong','Taipei','Tokyo','Beijing','Seoul']
city_list=[]

Cities={'name':'','lat':0,'lng':0}
for i in names:
    location = geolocator.geocode(i)
    Cities['name']=i
    Cities['lat']=location.latitude
    Cities['lng']=location.longitude
    city_list.append(dict(Cities))
  

In [4]:
city_list

[{'name': 'Kuala Lumpur', 'lat': 3.1516964, 'lng': 101.6942371},
 {'name': 'Jakarta', 'lat': -6.1753942, 'lng': 106.827183},
 {'name': 'Bangkok', 'lat': 13.7542529, 'lng': 100.493087},
 {'name': 'Hong Kong', 'lat': 22.2793278, 'lng': 114.1628131},
 {'name': 'Taipei', 'lat': 25.0375198, 'lng': 121.5636796},
 {'name': 'Tokyo', 'lat': 35.6828387, 'lng': 139.7594549},
 {'name': 'Beijing', 'lat': 39.906217, 'lng': 116.3912757},
 {'name': 'Seoul', 'lat': 37.5666791, 'lng': 126.9782914}]

Part 1(Marking the location of each city in the map using folium)

In [9]:
m=folium.Map(location=(city_list[0].get('lat'),city_list[0].get('lng')))
for i in range(len(city_list)):
    folium.Marker(location=(city_list[i].get('lat'),city_list[i].get('lng')),popup=city_list[i].get('name'),tooltip=city_list[i].get('name')).add_to(m)
 
m


In [10]:
def dis(A=list()):
    distances = list()
    dic = {}
    for i in range(0,len(A),1):
        for b in range(0,len(A),1):
            p1=(A[i].get('lat'),A[i].get('lng'))
            p2=(A[b].get('lat'),A[b].get('lng'))
            dic ['distance'] = int(geodesic(p1,p2).miles)
            dic ['place'] = A[i].get('name') +' to ' +A[b].get('name')
            distances.append(dict(dic))

    return distances   

def algo(start ,names):
    lenght= len(names)
    mat1=np.zeros((lenght,lenght))
    distance_matrix =mat1
    a=dis(city_list)
    for x in range(0,lenght,1):
        for y in range(lenght):
            b= a.pop()
            mat1[x,y]=b['distance']

   
    
    start = len(names)-start-1
    distance_matrix =mat1
    print(mat1)
    return heldKarp(start,mat1 ) , mat1



import sys
def findMinCost(m ,route):
    cost = 0
    for i in range(0,len(route)-1,1):
        x = route[i]
        y = route[i+1]
        print(y)
        cost+= m[x][y]
    return cost
def heldKarp(S , m ):
    N=len(m[0])
    memo = [[0 for i in range(2**N)] for j in range(N)]
    memo =  setup(m , memo , S , N )

    solve(m , memo , S , N)
    value = findOptimalTour(m , memo , S , N)
    cost = findMinCost(m , value)
    print(f'the min cost is {cost}')
    return value

def com1(r, N):
    subsets = list()
    com(0,0,r,N,subsets)
    return subsets
def com(set, at, r,n,subsets = list()):
    elp = n-at
    if elp<r:
        return
    if r == 0:
        subsets.append(set)
    else:
        for i in range(at , n , 1):
            set |= (1<<i)
            com(set , i+1 , r-1 , n , subsets)
            set = set & ~(1 << i)

def notin(ele , subset):
    return (((1<<ele) & subset) == 0)

def solve(m , memo , S , N): 
    for r in range(3, N+1 , 1):
        for subsets in com1(r , N):
            if notin(S , subsets):
                continue
            for next in range(N):
                if next==S or notin(next,subsets):
                    continue
                state=subsets^(1<<next)
                minDist=sys.maxsize
                for e in range(N):
                    if e==S or e==next or notin(e,subsets):
                        continue
                    newDistance=memo[e][state]+m[e][next]
                    if newDistance<minDist:
                        minDist=newDistance
                memo[next][subsets]=minDist 
                    


def setup(m , memo , S , N ):
    for i in range(N):
        if i == S:
            continue
        memo[i][1<<S|1<<i] = m[S][i]
    return memo

def findOptimalTour(m , memo , S , N):
    last = S
    state  = (1<<N)-1
    tour =[0 for i in range(N+1)]
    for i in range(N-1 , 0 , -1):
        index =-1
        for j in range(N):
            if j==S or notin(j,state):
                continue
            if index==-1:
                index =j
            prevDist = memo[index][state]+m[index][last]
            newDist = memo[j][state]+m[j][last]
            if newDist< prevDist:
                index=j
        tour[i] = index
        state = state^(1<<index)
        last=index
    tour[0] = tour[N] = S
    return tour
route , distance_matrix  = algo(0 , names)
print(route)
citylist=[]
index_map = dict()
def displayRoute(route , cityNames = list()):
    
    cityNames = cityNames[-1::-1]
    for r in route:
           citylist.append(cityNames[r])
           index_map[r]=cityNames[r]
    print(citylist)
displayRoute(route , names)


[[   0.  593.  721.  920. 1300. 2311. 3277. 2859.]
 [ 593.    0. 1307. 1067. 1221. 2043. 3228. 2691.]
 [ 721. 1307.    0. 1307. 1795. 2864. 3587. 3304.]
 [ 920. 1067. 1307.    0.  505. 1575. 2363. 2003.]
 [1300. 1221. 1795.  505.    0. 1072. 2017. 1558.]
 [2311. 2043. 2864. 1575. 1072.    0. 1436.  733.]
 [3277. 3228. 3587. 2363. 2017. 1436.    0.  732.]
 [2859. 2691. 3304. 2003. 1558.  733.  732.    0.]]
6
4
3
2
0
1
5
7
the min cost is 8651.0
[7, 6, 4, 3, 2, 0, 1, 5, 7]
['Kuala Lumpur', 'Jakarta', 'Hong Kong', 'Taipei', 'Tokyo', 'Seoul', 'Beijing', 'Bangkok', 'Kuala Lumpur']


In [11]:
marker_location=[]
for j in citylist:
        location = geolocator.geocode(j)
        marker_location.append((location.latitude,location.longitude))   
folium.PolyLine(marker_location).add_to(m)
m 


# Problem2 (Scraping the data from webpages and pickling them for later use)

In [12]:
raw_d = {}
# Run these sraping Parts only one time in your pc 
def Jak_data():
    url = [
        'https://www.thejakartapost.com/news/2020/05/06/bi-sees-growth-bottoming-in-q2-pledges-to-use-all-instruments-to-bolster-economy.html/',
        'https://www.thejakartapost.com/news/2020/05/06/indonesias-economy-heads-into-turbulence-as-q1-growth-plunges-economists.html/',
        'https://www.thejakartapost.com/news/2020/05/05/indonesias-q1-economic-growth-the-weakest-since-2001-slows-more-than-expected.html/',
        'https://www.thejakartapost.com/news/2020/04/14/imf-projects-0-5-growth-for-indonesia-as-global-economy-faces-deep-recession.html/',
        'https://www.thejakartapost.com/news/2020/04/01/indonesias-economy-may-contract-04-in-worst-case-scenario-sri-mulyani.html/',
    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(class_="col-md-10 col-xs-12 detailNews")
        article = [para.text for para in body.find_all('p')] #finds all paras and each para as an element of the array
        data.append(article)

    return data

In [13]:
def Bang_data():
    url = ['https://www.thailand-business-news.com/economics/79258-thai-economy-shrinks-1-8-in-first-quarter.html',
           'https://www.thailand-business-news.com/banking/79292-bot-cut-rate-to-record-low-0-5.html',
           'https://www.thailand-business-news.com/economics/79161-jsccib-expects-thai-economy-to-contract-by-3-5-this-year.html',
           'https://www.thailand-business-news.com/banking/78962-thai-economy-contracted-at-a-higher-rate-in-march.html'
           'https://www.thailand-business-news.com/economics/78696-march-inflation-hits-51-month-low-at-0-54-percent.html'

    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(id = "mvp-content-main")
        article = [para.text for para in body.find_all('p')]  # finds all paras and each para as an element of the array
        data.append(article)

    return data

In [14]:
def Tai_data():
    url = [
    'https://www.taipeitimes.com/News/biz/archives/2020/05/23/2003736872',
        'https://www.taipeitimes.com/News/biz/archives/2020/05/23/2003736867',
        'https://www.taipeitimes.com/News/editorials/archives/2020/05/17/2003736540',
        'https://www.taipeitimes.com/News/biz/archives/2020/05/19/2003736644',
        'https://www.taipeitimes.com/News/biz/archives/2020/05/19/2003736646'
    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(class_="archives")
        article = [para.text for para in body.find_all('p')]  # finds all paras and each para as an element of the array
        data.append(article)

    return data

In [15]:
def Hk_data():

    url = [
        'https://hongkongfp.com/2020/05/05/hong-kong-economy-shrinks-record-8-9-in-first-quarter-amid-coronavirus-worst-on-record/',
        'https://hongkongfp.com/2020/04/26/with-hong-kong-no-longer-the-worlds-freest-economy-is-it-time-to-rethink-the-capital-of-capitalism/',
        'https://hongkongfp.com/2020/03/10/hong-kong-low-income-families-face-unemployment-economy-slumps-amid-coronavirus-outbreak/',
        'https://hongkongfp.com/2020/03/18/govt-blames-protests-hong-kong-loses-worlds-freest-economy-accolade-25-years/',
        'https://hongkongfp.com/2020/05/04/hongkongers-support-of-pro-democracy-shops-violates-the-free-market-says-beijing/'

    ]
    data = []
    for p in url:
        page = requests.get(p)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(class_="entry-content")
        article = [para.text for para in body.find_all('p')] #finds all paras and each para as an element of the array
        data.append(article)

    return data

In [16]:
def Tok_data():
    url = ['https://www.japantimes.co.jp/news/2020/05/22/business/economy-business/japan-finance-boj-chiefs-vow-rare-meeting-use-tools-combat-virus-fallout/#.XsgY6zozZPY',
            'https://www.japantimes.co.jp/news/2020/05/22/business/economy-business/japan-core-consumer-prices-fall/#.XsgZHzozZPY',
           'https://www.japantimes.co.jp/news/2020/05/22/business/economy-business/boj-negative-inflation/#.XsgaZTozZPY',
           'https://www.japantimes.co.jp/news/2020/05/01/business/economy-business/deflation-pandemic-consumer-prices/#.XsgamDozZPY',
           'https://www.japantimes.co.jp/news/2020/04/24/business/economy-business/slowdown-inflation-deflation/#.XsgamDozZPY'
    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(class_="entry")
        article = [para.text for para in body.find_all('p')]  # finds all paras and each para as an element of the array
        data.append(article)

    return data

In [17]:
def Bei_data():
    url = [
        'https://news.cgtn.com/news/2020-05-22/Experts-China-right-to-rebalance-economic-targets-post-COVID-19-QHIlGgUqU8/index.html',
        'https://newsaf.cgtn.com/news/2020-05-22/China-to-accomplish-development-goals-by-observing-laws-of-economics-QI04IMVdaE/index.html',
        'https://news.cgtn.com/news/2020-05-22/China-prioritizes-employment-over-GDP-goal-experts--QHoTeYkQFy/index.html',
       'https://news.cgtn.com/news/2020-05-22/China-pledges-to-boost-consumption-and-expand-investment-QH33jJ5tu0/index.html',
        'https://news.cgtn.com/news/2020-05-22/China-to-implement-macro-policies-to-keep-business-employment-stable-QH3ci8hBxS/index.html'


    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(id ="cmsMainContent")
        article = [para.text for para in body.find_all('p')]  # finds all paras and each para as an element of the array
        data.append(article)

    return data

In [18]:
def Seo_data():
    url = [
    'http://english.chosun.com/site/data/html_dir/2020/04/10/2020041000745.html?related_all',
    'http://english.chosun.com/site/data/html_dir/2020/05/04/2020050401825.html?related_all',
    'http://english.chosun.com/site/data/html_dir/2020/05/21/2020052101932.html?related_all',
    'http://english.chosun.com/site/data/html_dir/2020/04/24/2020042401897.html',
    'http://english.chosun.com/site/data/html_dir/2020/05/08/2020050802097.html?related_all'
    ]
    data = []
    for pg in url:
        page = requests.get(pg)
        soup = BeautifulSoup(page.content, "html.parser")
        body = soup.find(id ="news_body_id")
        article = [para.text for para in body.find_all('p')]  # finds all paras and each para as an element of the array
        data.append(article)

    return data

In [20]:
raw_d['Jakarta'] = Jak_data()
raw_d['Bangkok'] = Bang_data()
raw_d['Taipei'] = Tai_data()
raw_d['Hong Kong'] = Hk_data()
raw_d['Tokyo'] = Tok_data()
raw_d['Beijing'] = Bei_data()
raw_d['Seoul'] = Seo_data()

In [21]:
with open('my_pickle', 'wb') as pickle_data:
    pickle.dump(raw_d, pickle_data)

with open('my_pickle', 'rb') as pickle_data:
    new_data = pickle.load(pickle_data)
print(new_data)
# after this comment out all the above cell

{'Jakarta': [['Bank Indonesia (BI) has pledged to go all out to bolster the economy as the central bank has forecast economic growth to weaken 0.4 percent in the second quarter due to coronavirus containment measures.', 'BI Governor Perry Warjiyo projected gross domestic product (GDP) growth to pick up slightly to 1.2 percent in the third quarter and 3.1 percent in the fourth quarter, adding that the central bank might revise down its initial growth projection of 2.3 percent this full year after lower-than-expected first quarter growth.', 'BI’s forecast was based on the assumption that the virus outbreak would peak in May until mid-June, with economic activities starting to stabilize in the third quarter.', '“Our stance remains loose, we will use all of our instruments to boost the economy,” Perry told reporters during a livestreamed news conference on Wednesday. “In the short term, our interest rate policy will prioritize rupiah stability, although there is room to lower the rate.”', 

NOW IN THIS PART , WE'RE MAKING THE DATA READY FOR MAKING DATAFRAME

In [23]:
with open('my_pickle', 'rb') as unpickle:
    new_data = pickle.load(unpickle) #opening previously pickled file

def array_combiner(array):
    x = sum(array, [])
    return x

data_array_combined = {key: array_combiner(value) for (key, value) in new_data.items()} #combining the arrays


def combine_text(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text


data_combined = {key: [combine_text(value)] for (key, value) in data_array_combined.items()}

T=str(data_combined['Taipei'])
B=str(data_combined['Bangkok'])
J=str(data_combined['Jakarta'])
H=str(data_combined['Hong Kong'])
To=str(data_combined['Tokyo'])
Be=str(data_combined['Beijing'])
S=str(data_combined['Seoul'])

NOW WE MAKE OUR DATA FRAME

In [24]:
data={'City':['Jakarta','Bangkok','Hong Kong','Taipei','Tokyo','Beijing','Seoul'],'Text':[J,B,H,T,To,Be,S]}

In [25]:
df=pd.DataFrame(data)
df.head(7)

,City,Text
0,Jakarta,"[""Bank Indonesia (BI) has pledged to go all ou..."
1,Bangkok,['The Thai economy shrank by 1.8 per cent year...
2,Hong Kong,['Hong Kong suffered its worst quarterly contr...
3,Taipei,['Public confidence in Taiwan’s economy and eq...
4,Tokyo,['Finance Minister Taro Aso and Bank of Japan ...
5,Beijing,['China has made the right move by altogether ...
6,Seoul,"['\r\n\tApril 10, 2020 09:06 \r\n The Korea Ec..."


CLEANING THE DATA FOR THE REST OF THE PART 

In [26]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text1=str(text)
    text1 = text1.lower()
    text1 = re.sub(r'https?://\S+', '', text1)
    text1 = re.sub(r'[^\w\s]','',text1)
    text1 = re.sub(r"\s+[a-zA-Z]\s+", " ", text1)
    text1 = re.sub("\d+", "", text1)
    text1 = re.sub(r"\s+"," ", text1, flags = re.I)
    text1 = re.sub(r"^\s+", "", text1)
    text1 = text1.rstrip("\n")
    text1 = re.sub('-',' ',text1)
    text1 = re.sub('_',' ',text1)
    text1=re.sub("([^\x00-\x7F])+"," ",text1)
    return text1


In [27]:
df['Text']=df['Text'].apply(clean_text)
df
# now the data is clean and is ready for algorithm part to remove stop word

,City,Text
0,Jakarta,bank indonesia bi has pledged to go all out to...
1,Bangkok,the thai economy shrank by per cent year on ye...
2,Hong Kong,hong kong suffered its worst quarterly contrac...
3,Taipei,public confidence in taiwans economy and equit...
4,Tokyo,finance minister taro aso and bank of japan go...
5,Beijing,china has made the right move by altogether dr...
6,Seoul,rntapril rn the korea economic research instit...


Problem2 (COUNTING WORD FREQUENCY FOR STOP WORDS AND REST OF THE WORD ) USING CountVectorizer

In [28]:
# first I'm finding the stop words from data
def stop_words_list(text):
    text1=str(text)
    stop_words_array=[]
    for i in text1.split(' '):
        for j in range(len(stop_words)):
            if i == stop_words[j]:
                stop_words_array.append(stop_words[j])
        
    return str(stop_words_array)    

In [29]:
stopwords=df['Text'].apply(stop_words_list)

In [30]:
# now with sklearn we're counting 
from sklearn.feature_extraction.text import CountVectorizer
# this one is for counting all the words except stop words
cv = CountVectorizer(stop_words='english')
# using cv2 to count stop words
cv2=CountVectorizer()

In [31]:
stop_cv=cv2.fit_transform(stopwords)
data_cv = cv.fit_transform(df['Text'])

MAKING DATAFRAME TO SHOW WORD FREQUENCIES

In [32]:
# first all the stop words
data_stop = pd.DataFrame(stop_cv.toarray(), columns=cv2.get_feature_names())
data_stop

,about,above,after,again,against,all,am,an,and,any,...,where,which,while,who,why,will,with,won,you,your
0,1,0,4,0,2,5,0,1,50,0,...,1,4,4,0,0,14,8,0,0,0
1,1,0,2,0,0,0,0,0,29,0,...,0,1,0,0,0,5,4,0,0,0
2,1,0,9,1,3,1,0,19,83,3,...,1,10,5,6,2,6,17,0,0,0
3,2,1,3,1,2,1,0,7,53,1,...,2,6,4,1,0,2,23,1,0,0
4,12,0,7,0,1,2,0,17,58,1,...,0,12,4,0,0,12,14,0,5,0
5,6,1,2,3,1,14,1,10,119,1,...,1,3,4,0,0,56,13,0,3,1
6,0,0,1,1,0,2,0,1,41,0,...,1,4,7,0,0,15,2,0,0,0


In [33]:
# second other words
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm

,aa,aaditya,abdur,abe,abenomics,abes,ability,able,aboutturn,abroad,...,yoy,yu,yuan,yuanliou,yuki,zealand,zero,zeroemployment,zone,zones
0,0,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,3,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,2,3,2,0,0,0,0,...,0,0,0,0,1,0,2,0,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,1,13,0,0,0,0,1,0,1
6,1,0,0,0,0,0,0,2,1,1,...,0,0,0,0,0,0,1,0,0,0


In [34]:
# scatter plot for stop words
data_stop.iplot(kind='scatter',mode='markers',size=10)

In [71]:
# scatter plot for others
data_dtm.iplot(kind='scatter',mode='markers',size=10)

Problem2 (Removing stop words by using RABIN KARP ALGORITHM)

In [72]:
# list of stop words
patdict = {1: ['a', 'i'],
           2: ['aa', 'am', 'an', 'as', 'at', 'be', 'by', 'do', 'he', 'id', 'im', 'if', 'in', 'is', 'it', 'me', 'my',
               'no', 'of', 'on', 'or', 'so', 'to', 'up', 'we'],
           3: ['all', 'and', 'any', 'are', 'but', 'did', 'few', 'for', 'had', 'has', 'hed', 'hes', 'her', 'him', 'his',
               'how', 'ill', 'ive', 'its', 'its', 'nor', 'not', 'off', 'our', 'out', 'own', 'she', 'the', 'too', 'was',
               'wed', 'who', 'why', 'you'],
           4: ['been', 'both', 'cant', 'does', 'dont', 'down', 'each', 'from', 'have', 'hell', 'here', 'hers', 'hows',
               'into', 'isnt', 'lets', 'more', 'most', 'once', 'only', 'ours', 'over', 'same', 'shed', 'shes', 'some',
               'such', 'than', 'that', 'them', 'then', 'they', 'this', 'very', 'well', 'were', 'weve', 'were', 'what',
               'when', 'whos', 'whom', 'whys', 'with', 'wont', 'youd', 'your','will','since'],
           5: ['about', 'above', 'after', 'again', 'arent', 'being', 'below', 'could', 'didnt', 'doing', 'hadnt',
               'hasnt', 'heres', 'other', 'ought', 'shant', 'shell', 'thats', 'their', 'there', 'these', 'theyd',
               'those', 'under', 'until', 'wasnt', 'whats', 'whens', 'where', 'which', 'while', 'would', 'youll',
               'youre', 'youve', 'yours'],
           6: ['before', 'cannot', 'doesnt', 'during', 'havent', 'having', 'itself', 'mustnt', 'myself', 'should',
               'theirs', 'theres', 'theyll', 'theyre', 'theyve', 'werent', 'wheres'],
           7: ['against', 'because', 'between', 'couldnt', 'further', 'herself', 'himself', 'through', 'wouldnt'],
           8: ['shouldnt', 'yourself','although']}

In [73]:
def find_hashValue(seq, paternLength, base, q):
    hashval = 0  # setting the initial hash val as 0

    for i in range(paternLength):  # we need to compare 3 letters only and thus the pat len is taken
        hashval = (hashval * base + ord(seq[i])) % q
    return hashval


def find_hashValues(seq, base, q):
    patternDict = {}
    for x in seq:
        hashval = 0
        for i in range(len(x)):
            hashval = (hashval * base + ord(x[i])) % q
        patternDict[hashval] = x
    return patternDict


def rabkar_search(txt, pat, base, q, patternLength):
    new_txt = txt
    len_txt = len(txt)
    len_pat = patternLength
    h = pow(base,len_pat-1) % q
    txt_hash = find_hashValue(txt, patternLength, base, q)
    pat_hash = find_hashValues(pat, base, q)
    result = []  # creating an empty list to store value
    deficit=0
    for position in range(len_txt - len_pat + 1):
        if txt_hash in pat_hash:
            match = True
            lastindex = 0
            
            for k in range(len_pat):
                x = pat_hash[txt_hash]
                if x[k] != txt[position + k]:  # in order to traverse along the text we do pos+k
                    match = False
                    break
                lastindex = k
            if (len(txt) > lastindex + position + 1):
                if (txt[lastindex + position + 1] != " "):
                    match = False
            if position != 0:
                if txt[position - 1] != " ":
                    match = False
            if match:  # i.e match = true
                new_txt =new_txt[:position-deficit]+new_txt[position-deficit+lastindex+2:]
                deficit =deficit+len(x)+1
                print(new_txt)
        if position < len_txt - len_pat:
            txt_hash = (txt_hash - h * ord(txt[position])) % q
            txt_hash = ((txt_hash * base) + ord(txt[position + len_pat])) % q
            txt_hash = (txt_hash + q) % q
    return new_txt


def rabinKarp(text,patdict):
    a = []
    length = []
    q=997
    base=26
    for i in range(1, 9, 1):
        text = rabkar_search(text, patdict[i], base, q, i)
    return text

In [74]:
# now we update the data
df['Text']=df['Text'].apply(lambda i :  rabinKarp(i,patdict))

In [75]:
df


,City,Text
0,Jakarta,bank indonesia bi pledged go bolster economy c...
1,Bangkok,thai economy shrank per cent year year first q...
2,Hong Kong,hong kong suffered worst quarterly contraction...
3,Taipei,public confidence taiwans economy equity inves...
4,Tokyo,finance minister taro aso bank japan gov haruh...
5,Beijing,china made right move altogether dropping annu...
6,Seoul,rntapril rn korea economic research institute ...


Problem2 (Finding POSITIVE AND NEGATIVE WORDS BY USING AHO-CORASICK ALGORITHM)

In [76]:
# I need to access the data so I read from csv file I saved before
# d=pd.read_csv('C:\\Algo\\cleaned_data')
# in this part we're using a dictionary built-in corpus in NLTK Library 
dic=nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon as op

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [77]:
positive_words=list(op.positive())
print(len(positive_words))
negative_words=list(op.negative())
print(len(negative_words))

2006
4783


In [78]:
class AhoNode:
    def __init__(self):
        self.goto = {}
        self.out = []
        self.fail = None


def aho_create_forest(patterns):
    root = AhoNode()

    for path in patterns:
        node = root
        for symbol in path:
            node = node.goto.setdefault(symbol, AhoNode())
        node.out.append(path)   
        
    return root


def aho_create_statemachine(patterns):
    root = aho_create_forest(patterns)
    queue = []
    for node in root.goto.values():
        queue.append(node)
        node.fail = root
     
    while len(queue) > 0:
        rnode = queue.pop(0)

        for key, unode in rnode.goto.items():
            queue.append(unode)
            fnode = rnode.fail
            while fnode != None and fnode not in fnode.goto.keys():
                fnode = fnode.fail
            unode.fail = fnode.goto[key] if fnode else root
            unode.out += unode.fail.out

    return root


def aho_find_all(s, root):
    node = root
    array=[]
    for i in range(len(s)):
        while node != None and s[i] not in node.goto.keys():
            node = node.fail
        if node == None:
            node = root
            continue
        node = node.goto[s[i]]
        for pattern in node.out:
            array.append(pattern)
    return array

In [79]:
list_word=[]
dict_words={'City':'','Positive Words':'','Negative Words':''}
pos_root = aho_create_statemachine(positive_words)
neg_root=aho_create_statemachine(negative_words)
def finding(country,text):
    dict_words['City']=country
    dict_words['Positive Words']=aho_find_all(text,pos_root)
    dict_words['Negative Words']=aho_find_all(text,neg_root)
    list_word.append(dict(dict_words))

In [80]:
for i in range(len(df['City'])):
    finding(df['City'][i],df['Text'][i])

In [81]:
word_data=pd.DataFrame(list_word)  

In [82]:
word_data

,City,Positive Words,Negative Words
0,Jakarta,"[bolster, stabilize, boost, stability, recover...","[weak, weaken, gross, virus, outbreak, stab, l..."
1,Bangkok,"[like, better, bloom, effective, support, reco...","[din, ding, gross, worst, decline, sin, crisis..."
2,Hong Kong,"[modern, ready, helped, bloom, intelligence, i...","[suffer, suffered, worst, sin, beg, outbreak, ..."
3,Taipei,"[confidence, improve, improved, improve, impro...","[pan, din, ding, pan, dent, dents, deter, dete..."
4,Tokyo,"[facilitate, stabilize, heal, fun, like, work,...","[pan, fall, fallout, pan, issue, stab, mar, bl..."
5,Beijing,"[right, heal, premier, modest, wise, lead, qui...","[mad, lie, lier, sin, beg, swelling, dire, mar..."
6,Seoul,"[modern, like, sharp, aver, led, support, supp...","[tank, negative, shock, crisis, cons, screech,..."


In [83]:
word_data['Positive Words'].iplot(kind='histogram',size=24,barmode='group')

In [45]:
word_data['Negative Words'].iplot(kind='histogram',size=12,barmode='group')

Problem2 (sentiment analysis)

In [84]:
def find_rank(list,key):
    result = [p[key] for p in list_word] 
    pos_num = [len(i) for i in result]
    return pos_num

In [85]:
df['Positive occurences'] = find_rank(list_word,'Positive Words')
df['Negative occurences'] = find_rank(list_word, 'Negative Words')

In [86]:
df

,City,Text,Positive occurences,Negative occurences
0,Jakarta,bank indonesia bi pledged go bolster economy c...,87,188
1,Bangkok,thai economy shrank per cent year year first q...,30,62
2,Hong Kong,hong kong suffered worst quarterly contraction...,203,400
3,Taipei,public confidence taiwans economy equity inves...,87,171
4,Tokyo,finance minister taro aso bank japan gov haruh...,102,370
5,Beijing,china made right move altogether dropping annu...,193,223
6,Seoul,rntapril rn korea economic research institute ...,47,122


In [87]:
df['Sentiment']=(df['Positive occurences']-df['Negative occurences'])

In [88]:
df['Sentiment_score']=df['Sentiment']/len(df['Text'][0].split(' '))*100
# we used this method as a percentage to be used in  the next problem

In [89]:
df

,City,Text,Positive occurences,Negative occurences,Sentiment,Sentiment_score
0,Jakarta,bank indonesia bi pledged go bolster economy c...,87,188,-101,-6.728847
1,Bangkok,thai economy shrank per cent year year first q...,30,62,-32,-2.131912
2,Hong Kong,hong kong suffered worst quarterly contraction...,203,400,-197,-13.124584
3,Taipei,public confidence taiwans economy equity inves...,87,171,-84,-5.596269
4,Tokyo,finance minister taro aso bank japan gov haruh...,102,370,-268,-17.854763
5,Beijing,china made right move altogether dropping annu...,193,223,-30,-1.998668
6,Seoul,rntapril rn korea economic research institute ...,47,122,-75,-4.996669


In [90]:
df.sort_values(by='Sentiment_score',ascending=False,kind='heapsort')

,City,Text,Positive occurences,Negative occurences,Sentiment,Sentiment_score
5,Beijing,china made right move altogether dropping annu...,193,223,-30,-1.998668
1,Bangkok,thai economy shrank per cent year year first q...,30,62,-32,-2.131912
6,Seoul,rntapril rn korea economic research institute ...,47,122,-75,-4.996669
3,Taipei,public confidence taiwans economy equity inves...,87,171,-84,-5.596269
0,Jakarta,bank indonesia bi pledged go bolster economy c...,87,188,-101,-6.728847
2,Hong Kong,hong kong suffered worst quarterly contraction...,203,400,-197,-13.124584
4,Tokyo,finance minister taro aso bank japan gov haruh...,102,370,-268,-17.854763


In [56]:
# df.to_csv('C:\\Algo\\Final_Data')

# Problem 3:

In [107]:
def satisfy(cityA , cityB):
    cityName = index_map[cityA]
    cityName2 = index_map[cityB]
    if cityName == "Kuala Lumpur" or cityName2 == "Kuala Lumpur":
        return True
    else:
        return float(df[df['City'] == cityName]["Sentiment_score"].values) - float(df[df['City'] == cityName2]["Sentiment_score"].values) >-2 and (1/(get_distance(cityA,cityB)+1))*100 <0.4
        
def get_distance(cityA , cityB):
    return distance_matrix[cityA , cityB]

In [108]:
possibleRoutes=[]
class Graph:

	# Constructor
    def __init__(self, adjList ,N):

       # A List of Lists to represent an adjacency list
        self.adjList = adjList
        


def printAllHamiltonianPaths(g, v, visited, path, N , distances):

	# if all the vertices are visited, then hamiltonian path exists
    if len(path) == N:
        
		# print hamiltonian path
        distances += get_distance(path[len(path)-1] , 7)
        tuplea =(path[:] ,distances)
        possibleRoutes.append(tuplea)
        return 

	# Check if every edge starting from vertex v leads to a solution or not
    for w in range(len(g.adjList[v])):

		# process only unvisited vertices as hamiltonian
		# path visits each vertex exactly once
        
        if not visited[int(w)]:
               if satisfy(path[len(path)-1] , w):
                    distances += get_distance(path[len(path)-1] , w) 
                    visited[w] = True
                    path.append(w)

			# check if adding vertex w to the path leads to solution or not
                    printAllHamiltonianPaths(g, w, visited, path, N , distances)

                    # Backtrack
                    visited[w] = False
                    distances -= get_distance(path[len(path)-2] , w) 
                    path.pop()

In [109]:
def merge(A=list()):
    left = A[:int(len(A) / 2)]
    right = A[int(len(A) / 2):]
    if len(A)==1:
        return A
    else:
        left=merge(left)
        right=merge(right)
    result=rs(left,right)
    return result
def rs(left,right):
    result=list()
    R=0
    L=0
    while(R<len(right) and L<len(left)):

        if left[L][1]<right[R][1]:
            result.append(left[L])
            L+=1
        else:
            result.append(right[R])
            R+=1
    result+=left[L:]
    result+=right[R:]
    return result


In [110]:
g = Graph(distance_matrix , len(distance_matrix))

In [111]:
N=8
start = 7
visited = [False] * N
visited[start] = True
path = [start]
distances = 0
printAllHamiltonianPaths(g ,start, visited , path , N , distances )

In [112]:
len(possibleRoutes)

12

In [113]:
merge(possibleRoutes)

[([7, 5, 1, 0, 3, 6, 4, 2], 13768.0),
 ([7, 5, 1, 0, 6, 3, 4, 2], 14613.0),
 ([7, 5, 1, 3, 0, 6, 4, 2], 15156.0),
 ([7, 5, 1, 6, 3, 0, 4, 2], 15686.0),
 ([7, 5, 1, 6, 0, 3, 4, 2], 15805.0),
 ([7, 1, 5, 6, 3, 0, 4, 2], 15852.0),
 ([7, 5, 1, 3, 6, 0, 4, 2], 15882.0),
 ([7, 1, 5, 6, 0, 3, 4, 2], 15971.0),
 ([7, 1, 5, 0, 3, 6, 4, 2], 17444.0),
 ([7, 1, 5, 3, 0, 6, 4, 2], 17622.0),
 ([7, 1, 5, 0, 6, 3, 4, 2], 18289.0),
 ([7, 1, 5, 3, 6, 0, 4, 2], 18348.0)]

In [114]:
def display(possible):
  for i in possible:
    for j in i[0]:
      print(index_map[j] ," -- ")
     
    print('\n')
display(possibleRoutes)

Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Seoul  -- 
Taipei  -- 
Jakarta  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Seoul  -- 
Jakarta  -- 
Taipei  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Taipei  -- 
Seoul  -- 
Jakarta  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Taipei  -- 
Jakarta  -- 
Seoul  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Jakarta  -- 
Seoul  -- 
Taipei  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Beijing  -- 
Bangkok  -- 
Jakarta  -- 
Taipei  -- 
Seoul  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Bangkok  -- 
Beijing  -- 
Seoul  -- 
Taipei  -- 
Jakarta  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Bangkok  -- 
Beijing  -- 
Seoul  -- 
Jakarta  -- 
Taipei  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Bangkok  -- 
Beijing  -- 
Taipei  -- 
Seoul  -- 
Jakarta  -- 
Hong Kong  -- 
Tokyo  -- 


Kuala Lumpur  -- 
Bangkok  -

In [65]:
print()